In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

# 45 years
#start_date = "1980-01-01"
#end_date = "2024-12-31"
# We have to use the date format that the WRDS database expects.
start_date = pd.Timestamp("1980-01-01")
end_date = pd.Timestamp("2024-12-31")



import sqlite3
# SQLite DB 파일 생성
# con = sqlite3.connect("tbtf.sqlite")
# database 연결
con = sqlite3.connect(database="../../wrds_80_24.sqlite")
# database에 dataframe 넣기
crsp_raw.to_sql(name="crsp_raw", con=con, if_exists="replace", index=False) # type: ignore

# 최종 DB 저장이 완료된 후, 배포·보관 목적의 최종 정리 단계에서, DB 파일 단위의 물리적 최적화
# tbtf.execute("VACUUM")

# 새로운 database를 연결해서 저장해 놓은 crsp dataframe을 확인
crsp_raw = pd.read_sql_query(
  sql="SELECT * FROM crsp_raw",
  con=con,
  parse_dates={"date"}
)
crsp_raw.head()

## WRDS

### Old. CRSP (monthly)


In [ ]:
import pandas as pd
import numpy as np

start_date = pd.Timestamp("2023-01-01")
end_date = pd.Timestamp("2024-12-31")

from sqlalchemy import create_engine

connection_string = (
    "postgresql+psycopg2://"
    "muppy23:pw"
    "@wrds-pgdata.wharton.upenn.edu:9737/wrds"
)

# Create the wrds engine object to access CRSP return data
wrds = create_engine(connection_string, pool_pre_ping=True)

crsp_monthly_query = (
  "SELECT msf.permno, msf.mthcaldt AS date, "
         "date_trunc('month', msf.mthcaldt)::date AS month, "
         "msf.mthret AS ret, msf.shrout, msf.mthprc AS altprc, "
         "msf.primaryexch, msf.siccd " # sic code from CRSP, not Compustat
    "FROM crsp.msf_v2 AS msf "
    "LEFT JOIN crsp.stksecurityinfohist AS ssih "
    "ON msf.permno = ssih.permno AND "
       "ssih.secinfostartdt <= msf.mthcaldt AND "
       "msf.mthcaldt <= ssih.secinfoenddt "
   f"WHERE msf.mthcaldt BETWEEN '{start_date}' AND '{end_date}' "
          "AND ssih.sharetype = 'NS' " # No Special, the share type of the security (COM, ADR, SBI)
          "AND ssih.securitytype = 'EQTY' " # Describes the type of a security (FUND, EQUTY, UNIT)
          "AND ssih.securitysubtype = 'COM' " #  the sub type of a security (COM, ETF, ETN)
          "AND ssih.usincflg = 'Y' " # if an issuer is incorporated in the US (Y = USINC, N = NONUS, X = N/A)
          "AND ssih.issuertype in ('ACOR', 'CORP')" # the type of the issuer of the security (REIT, CORP, LTD)
          #"AND msf.ticker = 'MSFT' " # sample ticker
)

crsp_monthly = (pd.read_sql_query(
    sql=crsp_monthly_query,
    con=wrds,
    dtype={"permno": int, "siccd": int},
    parse_dates={"date", "month"})
  .assign(shrout=lambda x: x["shrout"]*1000)
)
crsp_monthly.head()

In [ ]:

# mktcap
crsp_monthly = (crsp_monthly
  .assign(mktcap=lambda x: x["shrout"]*x["altprc"]/1000000)
  .assign(mktcap=lambda x: x["mktcap"].replace(0, np.nan))
)

# mktcap_lag
mktcap_lag = (crsp_monthly
  .assign(
    month=lambda x: x["month"]+pd.DateOffset(months=1),
    mktcap_lag=lambda x: x["mktcap"]
  )
  .get(["permno", "month", "mktcap_lag"])
)

crsp_monthly = (crsp_monthly
  .merge(mktcap_lag, how="left", on=["permno", "month"])
)

crsp_monthly = (crsp_monthly
  .dropna(subset=["mktcap", "mktcap_lag"])
)

### Old. Compustat (annual)

Take **all** 20 columns from the compustat table and create **uniform** random numbers between 0 and 1. For simplicity, we set the `datadate` for each firm-year observation to the last day of the year, although it is empirically not the case.

`comp.funda`  (Fundamentals Annual, quarterly update)
* identifiers : `gvkey` (firm identifier) + `datadate` (date identifier)
* `seq`, Stockholders Equity (total)
* `ceq`, Common Equity (total)
* `at`, Asset Total
* `lt`, Liability Total
* `txditc`, Deferred Taxes and Investment Tax Credit
* `txdb`, Deferred Taxes (Balance Sheet)
* `itcb`, Investment Tax Credit (balance sheet)
* `pstkrv`, Preferred stock Redemption Value
  * if missing, use `pstkl`, Liquidating Value
  * if still missing, use `pstk`, Preferred stock - Carrying Value, Stock (Capital)
* `capx`, Capital expenditures (Property, Plant & Equipment)
* `oancf`, Operating Activities - Net Cash Flow
* `sale`, Sales/Turnover (Net)
* `cogs`, Cost of Goods Sold
* `xint`, Interest On Deposits & Borrowing (total)
* `xsga`, Expenses of Selling, General and Administrative

Calculate
* `be`
* `op`
* `at_lag`,
* `inv`,


In [ ]:
compustat_query = (
  "SELECT gvkey, datadate, seq, ceq, at, lt, txditc, txdb, itcb,  pstkrv, "
         "pstkl, pstk, capx, oancf, sale, cogs, xint, xsga "
    "FROM comp.funda "
    "WHERE indfmt = 'INDL' "
          "AND datafmt = 'STD' "
          "AND consol = 'C' "
         f"AND datadate BETWEEN '{start_date}' AND '{end_date}'"
)

compustat_a = pd.read_sql_query(
  sql=compustat_query,
  con=wrds,
  dtype={"gvkey": str},
  parse_dates={"datadate"}
)

compustat_a = (compustat_a
  .assign(
    be=lambda x:
      (x["seq"].combine_first(x["ceq"]+x["pstk"])
       .combine_first(x["at"]-x["lt"])+
       x["txditc"].combine_first(x["txdb"]+x["itcb"]).fillna(0)-
       x["pstkrv"].combine_first(x["pstkl"])
       .combine_first(x["pstk"]).fillna(0))
  )
  .assign(
    be=lambda x: x["be"].apply(lambda y: np.nan if y <= 0 else y)
  )
  .assign(
    op=lambda x:
      ((x["sale"]-x["cogs"].fillna(0)-
        x["xsga"].fillna(0)-x["xint"].fillna(0))/x["be"])
  )
)

compustat_a = (compustat_a
  .assign(year=lambda x: pd.DatetimeIndex(x["datadate"]).year)
  .sort_values("datadate")
  .groupby(["gvkey", "year"])
  .tail(1)
  .reset_index()
)

compustat_lag = (compustat_a
  .get(["gvkey", "year", "at"])
  .assign(year=lambda x: x["year"]+1)
  .rename(columns={"at": "at_lag"})
)

compustat_a = (compustat_a
  .merge(compustat_lag, how="left", on=["gvkey", "year"])
  .assign(inv=lambda x: x["at"]/x["at_lag"]-1)
  .assign(inv=lambda x: np.where(x["at_lag"] <= 0, np.nan, x["inv"]))
)



### crsp_a_ccm

In [ ]:
# At present, your institution does not have access to crsp_a_ccm.
ccmxpf_linktable_query = (
  "SELECT lpermno AS permno, gvkey, linkdt, "
         "COALESCE(linkenddt, CURRENT_DATE) AS linkenddt "
    "FROM crsp.ccmxpf_linktable "
    "WHERE linktype IN ('LU', 'LC') "
          "AND linkprim IN ('P', 'C') "
          "AND usedflag = 1"
)

ccmxpf_linktable = pd.read_sql_query(
  sql=ccmxpf_linktable_query,
  con=wrds,
  dtype={"permno": int, "gvkey": str},
  parse_dates={"linkdt", "linkenddt"}
)

ccm_links = (crsp_monthly
  .merge(ccmxpf_linktable, how="inner", on="permno")
  .query("~gvkey.isnull() & (date >= linkdt) & (date <= linkenddt)")
  .get(["permno", "gvkey", "date"])
)

crsp_monthly = (crsp_monthly
  .merge(ccm_links, how="left", on=["permno", "date"])
)

(crsp_monthly
  .to_sql(name="crsp_monthly",
          con=tidy_finance, # type: ignore
          if_exists="replace",
          index=False)
)

## WRDS - merge by CUSIP
- `CRSP`와 `Compustat` 병합을 **CUSIP 기반**, **월간–연간 병합 (정렬된 panel window)** 방식으로 구현
- `merged`는 `crsp` 월별 수익률에 대해 최근 datadate 기준 Compustat 재무 정보가 병합된 결과
- `Compustat` 값은 해당 연도 말 기준으로 다음 회계년도까지 고정적으로 사용됨 (예: 2023-12-31 `datadate` → 2024년 월간 데이터까지 적용)
  - 2023년 Compustat 데이터(`datadate=2023-12-31`)는 2024년 월별 CRSP에 모두 복제되어 병합
- `CRSP`에는 `ticker` 추가 (stocknames)
- `Compustat`에는 `conm`, `tic` (회사명, 티커) 포함
- `date_trunc` 대신 SQL 표준 구문으로 `월말` datetime 통일


In [1]:
### 1. 데이터 수집 및 전처리
# pip install sqlalchemy
# pip install psycopg2-binary
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

# 기간 설정
start_date = pd.Timestamp("2023-01-01")
end_date = pd.Timestamp("2024-12-31")

# Create the wrds engine object to access CRSP return data
connection_string = (
    "postgresql+psycopg2://"
    "muppy23:pw"
    "@wrds-pgdata.wharton.upenn.edu:9737/wrds?sslmode=require"
)


# 1. 연결
# Create the wrds engine object to access CRSP return data
engine = create_engine(connection_string, pool_pre_ping=True)

# 3. 연결 해제
# engine.dispose()


In [ ]:
import wrds

# 자동으로 ~/.pgpass 또는 환경변수에서 비밀번호 읽음
db = wrds.Connection(wrds_username='muppy23')

In [ ]:
import psycopg2
import time

start = time.time()
try:
    conn = psycopg2.connect(
        dbname="wrds",
        user="muppy23",
        password="pw",
        host="wrds-pgdata.wharton.upenn.edu",
        port=9737
    )
    print("✅ 연결 성공")
except Exception as e:
    print("❌ 연결 실패:", e)
finally:
    end = time.time()
    print(f"연결 시간: {end - start:.2f}초")


In [ ]:
try:
    df_test = pd.read_sql("SELECT 1 AS connection_test", con=engine)
    print("WRDS 연결 성공 ✅")
    print(df_test)
except Exception as e:
    print("❌ 연결 실패:", e)


In [ ]:
# CRSP 월별 수익률 데이터 (with ticker)

# sic code from CRSP, not Compustat
# No Special, the share type of the security (COM, ADR, SBI)
# Describes the type of a security (FUND, EQUTY, UNIT)
#  the sub type of a security (COM, ETF, ETN)
# if an issuer is incorporated in the US (Y = USINC, N = NONUS, X = N/A)
# the type of the issuer of the security (REIT, CORP, LTD)
crsp_query = f"""
    SELECT 
        msf.permno,
        sn.cusip,
        msf.mthcaldt::date AS date,
        msf.mthret AS ret,
        msf.shrout,
        msf.mthprc AS altprc,
        msf.primaryexch,
        msf.siccd,
        sn.ticker
    FROM crsp.msf_v2 AS msf
    LEFT JOIN crsp.stksecurityinfohist AS ssih
        ON msf.permno = ssih.permno
        AND ssih.secinfostartdt <= msf.mthcaldt
        AND msf.mthcaldt <= ssih.secinfoenddt
    LEFT JOIN crsp.stocknames AS sn
        ON msf.permno = sn.permno
        AND msf.mthcaldt BETWEEN sn.namedt AND sn.nameenddt
    WHERE msf.mthcaldt BETWEEN '{start_date}' AND '{end_date}'
        AND ssih.sharetype = 'NS'
        AND ssih.securitytype = 'EQTY'
        AND ssih.securitysubtype = 'COM'
        AND ssih.usincflg = 'Y'
        AND ssih.issuertype IN ('ACOR', 'CORP')
"""
crsp_monthly = pd.read_sql_query(crsp_query, con=engine, parse_dates=["date"])
crsp_monthly["cusip8"] = crsp_monthly["cusip"].str[:8]
crsp_monthly.head()

In [ ]:
# Compustat 데이터 조회 (with gvkey, ticker, company name)
compustat_query = f"""
    SELECT gvkey, datadate, cusip, tic, conm,
           seq, ceq, at, lt, txditc, txdb, itcb, pstkrv,
           pstkl, pstk, capx, oancf, sale, cogs, xint, xsga
    FROM comp.funda
    WHERE indfmt = 'INDL' AND datafmt = 'STD' AND consol = 'C'
      AND datadate BETWEEN '{start_date}' AND '{end_date}'
"""

compustat = pd.read_sql_query(compustat_query, con=engine, parse_dates=["datadate"])
compustat.head()

In [44]:
# Compustat 파생변수 및 연도 매핑

# (1) 회계연도(fiscal year) 생성
compustat["fyear"] = compustat["datadate"].dt.year

# (2) Book equity 계산
compustat["be"] = (
    compustat["seq"]
    .combine_first(compustat["ceq"] + compustat["pstk"])
    .combine_first(compustat["at"] - compustat["lt"])
    + compustat["txditc"].combine_first(compustat["txdb"] + compustat["itcb"]).fillna(0)
    - compustat["pstkrv"].combine_first(compustat["pstkl"]).combine_first(compustat["pstk"]).fillna(0)
)

# (3) 음수 제거
compustat["be"] = compustat["be"].apply(lambda x: np.nan if x <= 0 else x)

# (4) OP (Operating Profitability)
compustat["op"] = (
    (compustat["sale"] - compustat["cogs"].fillna(0) - compustat["xsga"].fillna(0) - compustat["xint"].fillna(0))
    / compustat["be"]
)

# (5) Lagged total assets: 기준은 fyear
compustat_lag = (
    compustat[["gvkey", "fyear", "at"]]
    .assign(fyear=lambda x: x["fyear"] + 1)
    .rename(columns={"at": "at_lag"})
)

# (6) 병합 및 inv 계산
compustat = (
    compustat
    .merge(compustat_lag, on=["gvkey", "fyear"], how="left")
    .assign(
        inv=lambda x: np.where(
            x["at_lag"] <= 0, np.nan,
            x["at"] / x["at_lag"] - 1
        )
    )
)

# (7) CUSIP 병합용 필드
compustat["cusip8"] = compustat["cusip"].str[:8]

# at_lag가 없는 경우 (fyear - 1 데이터 없음) → 해당 firm-year 제거
compustat = compustat[~compustat["at_lag"].isna()].copy()


In [ ]:
# Compustat 확장: fyear + 12개월 월간 패널

# 월간 row로 확장
compustat_monthly_rows = []

for offset in range(12):
    df_temp = compustat.copy()
    # 회계연도 말일로부터 offset 개월 추가
    df_temp["date"] = df_temp["datadate"] + pd.DateOffset(months=offset + 1)
    compustat_monthly_rows.append(df_temp)

compustat_monthly = pd.concat(compustat_monthly_rows, ignore_index=True)

# CRSP와 병합 가능한 기간으로 필터링
compustat_monthly = compustat_monthly.query(
    "date >= @start_date and date <= @end_date"
).copy()

# 병합을 위한 cusip8 필드 정리
compustat_monthly["cusip8"] = compustat_monthly["cusip"].str[:8]

compustat_monthly.head()

In [ ]:
# 병합: cusip8 + date 기준
merged = crsp_monthly.merge(
    compustat_monthly,
    how="left",
    on=["cusip8", "date"],
    suffixes=('', '_comp')
)

merged.head(10)

In [ ]:
# 병합 후 진단
print("총 CRSP 레코드 수:", len(crsp_monthly))
print("Compustat 정보 병합 성공 비율:", merged["gvkey"].notnull().mean())


## End

In [48]:
# Compustat 월간 패널로 확장 (각 연도 datadate ~ 다음 12개월까지)
# 월간 index 생성
monthly_index = pd.date_range(start=start_date, end=end_date, freq="M")

# 연-월 조합 (각 datadate에 대해 12개월 확장)
panel_expand = []
for offset in range(12):
    df_temp = compustat.copy()
    df_temp["date"] = df_temp["datadate"] + pd.DateOffset(months=offset)
    panel_expand.append(df_temp)

compustat_monthly = pd.concat(panel_expand).query("date >= @start_date and date <= @end_date")
compustat_monthly.head()


gvkey   datadate      cusip   tic                         conm       seq  \
0  001004 2023-05-31  000361105   AIR                     AAR CORP  1099.100   
1  001004 2024-05-31  000361105   AIR                     AAR CORP  1189.800   
2  001045 2023-12-31  02376R102   AAL  AMERICAN AIRLINES GROUP INC -5202.000   
3  001045 2024-12-31  02376R102   AAL  AMERICAN AIRLINES GROUP INC -3977.000   
4  001050 2023-12-31  125141101  CECO      CECO ENVIRONMENTAL CORP   232.643   

        ceq         at       lt  txditc  ...      xint      xsga    cusip8  \
0  1099.100   1833.100    734.0  33.600  ...    12.200   220.000  00036110   
1  1189.800   2770.000   1580.2  23.900  ...    43.200   270.200  00036110   
2 -5202.000  63058.000  68260.0   9.000  ...  2145.000  7805.000  02376R10   
3 -3977.000  61783.000  65760.0   9.000  ...  1934.000  8241.000  02376R10   
4   232.643    600.291    362.8   8.838  ...    13.416   122.944  12514110   

   year        be        op   at_lag       inv  fyear       date  
0  2023  1132.700  0.147524      NaN       NaN   2023 2023-05-31  
1  2024  1213.700  0.142045   1833.1  0.511101   2024 2024-05-31  
2  2023       NaN       NaN      NaN       NaN   2023 2023-12-31  
3  2024       NaN       NaN  63058.0 -0.020219   2024 2024-12-31  
4  2023   241.481  0.162659      NaN       NaN   2023 2023-12-31  

[5 rows x 29 columns]

In [37]:
# CRSP–Compustat 병합 (CUSIP 기준)
merged = crsp_monthly.merge(
    compustat,
    how="left",
    left_on=["cusip8", "fyear"],
    right_on=["cusip8", "fyear"],
    suffixes=('', '_comp')
)

merged.head(20)

permno   cusip_x       date       ret   shrout    altprc primaryexch  \
0    10026  46603210 2023-01-31 -0.042816    19229  143.3000           Q   
1    10028  29402E10 2023-01-31  0.269924    26925    6.6798           A   
2    10032  72913210 2023-01-31 -0.067425    27702   95.9900           Q   
3    10044  77467X10 2023-01-31 -0.056439     6250    5.3783           Q   
4    10066  35518410 2023-01-31  0.141256    11784    5.0900           Q   
5    10104  68389X10 2023-01-31  0.086219  2696253   88.4600           N   
6    10107  59491810 2023-01-31  0.033317  7443804  247.8100           Q   
7    10138  74144T10 2023-01-31  0.067944   224310  116.4700           Q   
8    10145  43851610 2023-01-31 -0.027158   668140  208.4800           Q   
9    10158  02361E10 2023-01-31  0.128281    33914   64.4700           N   
10   10200  75991610 2023-01-31  0.094442    55558  185.3000           Q   
11   10220  05605H10 2023-01-31  0.047865    91222   60.8600           N   
12   10252  45383610 2023-01-31 -0.056141    45641   79.6900           Q   
13   10253  26253C20 2023-01-31  0.402439   139017    0.2300           A   
14   10257  23335Q10 2023-01-31  0.037505    13757   12.3100           Q   
15   10258  15117B20 2023-01-31 -0.011443    47201   44.0600           Q   
16   10294  41959620 2023-01-31  0.130120     1283   34.3500           N   
17   10308  89011010 2023-01-31 -0.029002    14484   75.3300           A   
18   10318  05766520 2023-01-31  0.069773    32153  130.6300           Q   
19   10333  98419J20 2023-01-31  0.102717    11454   20.2900           Q   

    siccd ticker    cusip8  ... oancf sale cogs xint xsga  year  be  op  \
0    2052   JJSF  46603210  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
1    5094    ELA  29402E10  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
2    3670   PLXS  72913210  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
3    2060   RMCF  77467X10  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
4    7373   FKWL  35518410  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
5    7379   ORCL  68389X10  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
6    7370   MSFT  59491810  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
7    6211   TROW  74144T10  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
8    3724    HON  43851610  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
9    1731   AMRC  02361E10  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
10   2830   RGEN  75991610  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
11   3511   BWXT  05605H10  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
12   6020   INDB  45383610  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
13   7373    DSS  26253C20  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
14   7363   DLHC  23335Q10  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
15   2835   CLDX  15117B20  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
16   5021    HVT  41959620  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
17   6022    TMP  89011010  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
18   2800   BCPC  05766520  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   
19   2834   XOMA  98419J20  ...   NaN  NaN  NaN  NaN  NaN   NaN NaN NaN   

    at_lag  inv  
0      NaN  NaN  
1      NaN  NaN  
2      NaN  NaN  
3      NaN  NaN  
4      NaN  NaN  
5      NaN  NaN  
6      NaN  NaN  
7      NaN  NaN  
8      NaN  NaN  
9      NaN  NaN  
10     NaN  NaN  
11     NaN  NaN  
12     NaN  NaN  
13     NaN  NaN  
14     NaN  NaN  
15     NaN  NaN  
16     NaN  NaN  
17     NaN  NaN  
18     NaN  NaN  
19     NaN  NaN  

[20 rows x 36 columns]